In [1]:
print("Hello")

Hello


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\smaran\anaconda3\envs\mchatbots\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "PINECONE_API_ENV = ""

In [3]:
#Extract data from the PDF
def load_PDF(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    
    return documents
    

In [4]:
extracted_data = load_PDF("data/")

In [5]:
#Create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("Length of my chunk:", len(text_chunks))


Length of my chunk: 7020


In [7]:
#Download Embedding model 
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2" )
    return embeddings

In [8]:
embeddings = download_hugging_face_embedding()

In [9]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [15]:
pinecone.init(api_key=PINECONE_API_KEY,environment = PINECONE_API_ENV)


In [16]:
index_name = "medical-chatbot"
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

In [17]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [19]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [20]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [22]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])